# SerpAPI Tool Helper

This notebook provides a quick rundown of how we construct an async SerpAPI tool. Let's start by query SerpAPI synchronously via their Python SDK:

In [1]:
from serpapi import GoogleSearch
from getpass import getpass

SERPAPI_API_KEY = getpass("Enter your SerpAPI API key: ")

params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

search = GoogleSearch(params)
results = search.get_dict()

print(results)

{'search_metadata': {'id': '688e7caf0d0b29b7d68996f4', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/ebbfa0286c652347/688e7caf0d0b29b7d68996f4.json', 'pixel_position_endpoint': 'https://serpapi.com/searches/ebbfa0286c652347/688e7caf0d0b29b7d68996f4.json_with_pixel_position', 'created_at': '2025-08-02 21:01:35 UTC', 'processed_at': '2025-08-02 21:01:35 UTC', 'google_url': 'https://www.google.com/search?q=latest+news+in+the+world&oq=latest+news+in+the+world&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/ebbfa0286c652347/688e7caf0d0b29b7d68996f4.html', 'total_time_taken': 7.4}, 'search_parameters': {'engine': 'google', 'q': 'latest news in the world', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'latest news in the world', 'total_results': 1990000000, 'time_taken_displayed': 0.3, 'organic_results_state': 'Results for exact spelling'}, 'related_questions': [{'question': 'What is the trending 

Our results are provided in the `"organic_results"` key:

In [2]:
results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjg4bjzgu2OAxUOFVkFHWWLFZ4QFnoECCIQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/688e7caf0d0b29b7d68996f4/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'World · Hamas refuses to disarm until Palestinian state established · Family condemns Hamas video showing emaciated Israeli hostage · Police find getaway car of ...',
  'snippet_highlighted_words': ['Hamas refuses to disarm until Palestinian state established'],
  'sitelinks': {'inline': [{'title': 'BBC World',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East',
     'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Africa', 'link': 'https://www.bb

We reformat this to extract only the most relevant information, such as the the title, source, link, and snippet. Let's use pydantic `BaseModel` to define this structure.

In [3]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    source: str
    link: str
    snippet: str

    @classmethod
    def from_serpapi_result(cls, result: dict) -> "Article":
        return cls(
            title=result["title"],
            source=result["source"],
            link=result["link"],
            snippet=result["snippet"],
        )

In [4]:
articles = [Article.from_serpapi_result(result) for result in results["organic_results"]]

In [6]:
articles[:9]  # Display the first 9 articles

[Article(title='World | Latest News & Updates', source='BBC', link='https://www.bbc.com/news/world', snippet='World · Hamas refuses to disarm until Palestinian state established · Family condemns Hamas video showing emaciated Israeli hostage · Police find getaway car of ...'),
 Article(title='World news - breaking news, video, headlines and opinion', source='CNN', link='https://www.cnn.com/world', snippet='View CNN world news today for international news and videos from Europe, Asia, Africa, the Middle East and the Americas.'),
 Article(title='World news', source='CBS News', link='https://www.cbsnews.com/world/', snippet='World · Pope thrills hundreds of thousands of young Catholics at youth festival · Team USA sets new relay record at swim worlds and Ledecky wins again · U.S. ...'),
 Article(title='Breaking News, World News and Video from Al Jazeera', source='Al Jazeera', link='https://www.aljazeera.com/', snippet='Al Jazeera · Siraj strikes after Jaiswal helps India set England daunt

## Using Async

All of this works, but it unfortunately is not async and the SerpAPI SDK does not support async either, so we much query the API directly using the `aiohttp` library.

In [7]:
import aiohttp

# Redefine the parameters for the asynchronous request
params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

# Uncomment the following lines to use synchronous requests
# requests.get(
#     "https://serpapi.com/search",
#     params=params
# )

async with aiohttp.ClientSession() as session:
    async with session.get(
        "https://serpapi.com/search",
        params=params
    ) as response:
        results = await response.json()

results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjg4bjzgu2OAxUOFVkFHWWLFZ4QFnoECCIQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/688e7caf0d0b29b7d68996f4/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'World · Hamas refuses to disarm until Palestinian state established · Family condemns Hamas video showing emaciated Israeli hostage · Police find getaway car of ...',
  'snippet_highlighted_words': ['Hamas refuses to disarm until Palestinian state established'],
  'sitelinks': {'inline': [{'title': 'BBC World',
     'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East',
     'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Africa', 'link': 'https://www.bb

With that, we have all we need to build a fully async serpapi tool.

In [8]:
from langchain_core.tools import tool

@tool
async def serpapi(query: str) -> list[Article]:
    """Use this tool to search the web."""
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": query,
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://serpapi.com/search",
            params=params
        ) as response:
            results = await response.json()
    return [Article.from_serpapi_result(result) for result in results["organic_results"]]

Note that because this tool is async, we cannot use `tool.func` to call it as before:

In [9]:
serpapi.func

Instead, we use `tool.coroutine`:

In [10]:
serpapi.coroutine

<function __main__.serpapi(query: str) -> list[__main__.Article]>

In [16]:
# how to manually call the async serpai tool

async def main():
    results = await serpapi.coroutine(input("Enter your search query: \n"))
    for article in results:
        print(f"Title: {article.title}\nSource: {article.source}\nLink: {article.link}\nSnippet: {article.snippet}\n")

await main()

Title: CA Hindus and Sikhs report record hate crimes, but why ...
Source: CalMatters
Link: https://calmatters.org/justice/2024/09/california-hindu-hate-crimes/
Snippet: The vandalization of three California Hindu temples escalated fears about hate crimes against Indian Americans. Meanwhile, Sikhs are on edge from attacks ...See more

Title: Anti-Hindu hate crimes rise in United States, again: FBI report
Source: Hindu American Foundation
Link: https://www.hinduamerican.org/press/anti-hindu-rate-crimes-united-states-rise-fbi-report/
Snippet: Newly released US government statistics show 14 cases of hate crimes targeting Hindu Americans in 2018 · 0.9% were anti-Hindu (14 offenses). · Hate crime murders ...See more

Title: Anti-Hindu sentiment
Source: Wikipedia
Link: https://en.wikipedia.org/wiki/Anti-Hindu_sentiment
Snippet: ... Hindu Americans have been the targets of bullying, discrimination, hate speech, harassment, and bias-motivated crimes." Criticism. Some academics ...See more

Titl

---